In [1]:
import os
import cv2
import pandas as pd
import numpy as np

In [2]:
from keras.preprocessing import image

In [3]:
categories=['with_mask','without_mask']

In [4]:
data=[]
for category in categories:
    path=os.path.join('facemask-dataset/train',category)
    label=categories.index(category)
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])
    

In [5]:
len(data)

1275

### Shuffling the current data

In [6]:
import random

In [7]:
random.shuffle(data)

### Seperating Labels and Features

In [8]:
X=[]
y=[]
for feature,label in data:
    X.append(feature)
    y.append(label)

### Length of the feature and label

In [9]:
print('Length of feature',len(X))
print('Length of label',len(y))

Length of feature 1275
Length of label 1275


In [10]:
# convert X and y to arrays

In [11]:
X=np.array(X)
y=np.array(y)

In [12]:
X.shape

(1275, 224, 224, 3)

### standardising X 

In [13]:
X=X/255

### splitting data for train and test

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [15]:
X_train.shape

(1020, 224, 224, 3)

In [16]:
X_test.shape

(255, 224, 224, 3)

In [17]:
from keras.applications.vgg16 import VGG16

In [18]:
vgg=VGG16()

In [19]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
from keras import Sequential

In [21]:
model=Sequential()

In [22]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
for layer in model.layers:
    layer.trainable=False

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [26]:
from keras.layers import Dense

In [27]:
model.add(Dense(1,activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [29]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
# model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [31]:
cap=cv2.VideoCapture(0)

In [32]:
# while True:
#     ret,frame=cap.read()
#     cv2.imshow('window',frame)
#     if cv2.waitKey(1) & 0xFF == ord('x'):
#         break


# cv2.destroyAllWindows()

In [34]:
import pickle

In [36]:
model=pickle.load(open('file.pkl','rb'))

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AddV2] name: 